# News classification (Bag of words) and sentiment analysis

# Importing libraries

In [1]:
from __future__ import print_function

import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
import csv
#import nltk
import string
import re
import sys

# Defining Classifier function 

In [2]:
def find_news_category(x) :
    class_output_NB = clf_NB.predict(x)
    class_description = " "
    if class_output_NB == '1':
        class_description = "World"
    if class_output_NB == '2':
        class_description = "Sports"
    if class_output_NB == '3':
        class_description = "Business"
    if class_output_NB == '4':
        class_description = "Sci/Tech"        
    print("@NB : This NEWS is related to : " + class_description)        
    class_output_svm = clf_svm.predict(x)
    class_description = " "
    if class_output_svm == '1':
        class_description = "World"
    if class_output_svm == '2':
        class_description = "Sports"
    if class_output_svm == '3':
        class_description = "Business"
    if class_output_svm == '4':
        class_description = "Sci/Tech"        
    print("@SVM : This NEWS is related to : " + class_description)       

# Reading the dataset

In [3]:
# read CSV file & load into list
with open("train.csv",'r') as my_file:
    reader = csv.reader(my_file, delimiter=',')
    news_data_train = list(reader)
    
with open("test.csv",'r') as my_file:
    reader = csv.reader(my_file, delimiter=',')
    news_data_test = list(reader)   

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

# Exploring the data 

In [98]:
print(news_data_train[3])

['3', 'Iraq Halts Oil Exports from Main Southern Pipeline (Reuters)', 'Reuters - Authorities have halted oil export\\flows from the main pipeline in southern Iraq after\\intelligence showed a rebel militia could strike\\infrastructure, an oil official said on Saturday.']


# Cleaning and combining the text data 

In [102]:
stopwords_file = open("stopwords.txt", "r" , encoding="utf8") 
stopwords = stopwords_file.read()
#stopwords.split()

In [103]:
#stop = set(stopwords.words('english'))

In [104]:
STOPWORD_customized = {'will','new','york','will' ,'quot','year ','company ','week', 'one','two','three','reuters','reuters' , 'monday','tuesday','wednesday','thursday','friday','saturday','sunday','yesterday'}

In [176]:
label = [] # class
text_doc = [] # news text 
c =0 # to control count of 
for i in news_data_train:
    c= c+1
    if c < 2000:
        label.append(i[0])            
        merged = i[1]+i[2] # combining header and content 
        lowers = merged.lower()   # to lower      
        no_punctuation = re.sub(r'[^\w\s]',' ', lowers)    # remove punctuation
        temp = ' '.join(word for word in no_punctuation.split() if len(word)>3) # remove samll words
        filtered = ' '.join(word for word in temp.split() if word not in stopwords.split()) # remove stopwords
        filtered = ' '.join(word for word in filtered.split() if word not in STOPWORD_customized) # remove stopwords
        text_doc.append(filtered)

In [177]:
print(text_doc[4])

prices soar time record posing menace economy tearaway prices toppling records straining wallets economic menace barely months presidential elections


In [178]:
#print(text_doc)
text = ""
for i in range (1,len(text_doc)):
    temp = (text_doc[i])
    text = text + temp

In [179]:
from collections import Counter
word_count = Counter()
word_count.update((word for word in text.split()))
#for word, count in word_count.most_common():
    #print(word, ":" ,count,"\n")      

In [4]:
#sys.getdefaultencoding()

In [183]:
from collections import Counter
#Counter(label).keys() # equals to list(set(words))
#Counter(label).values() # counts the elements' frequency
from sklearn.feature_extraction.text import CountVectorizer

In [184]:
vectorizer = CountVectorizer()
corpus = text_doc  # processed text corpous 
X = vectorizer.fit_transform(corpus)
X.shape

(49999, 69614)

In [185]:
#from sklearn.feature_extraction.text import TfidfTransformer
#tf_transformer = TfidfTransformer(use_idf=True).fit(X)
#X_tf = tf_transformer.transform(X)
#X_tf.shape

In [186]:
tfvectorizer = TfidfVectorizer(max_df=0.5, max_features= 10000, min_df=2, stop_words='english',use_idf= True)
X_tfidf = tfvectorizer.fit_transform(corpus)
#X_tfidf

<49999x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 761738 stored elements in Compressed Sparse Row format>

In [187]:
#X_tfidf[1].toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [188]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

In [189]:

print("Performing dimensionality reduction using LSA")
t0 = time()
# Vectorizer results are normalized, which makes KMeans behave as
# spherical k-means for better results. Since LSA/SVD results are
# not normalized, we have to redo the normalization.
svd = TruncatedSVD(1000)
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X_dim_reduced = lsa.fit_transform(X)

print("done in %fs" % (time() - t0))

Performing dimensionality reduction using LSA
done in 255.222253s


In [190]:
explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(
int(explained_variance * 100)))

print()

Explained variance of the SVD step: 54%



# Classification (NB)

In [163]:
# X -> full corpus
# X_tfidf -> TFIDF reduced 

In [191]:
# X  has text data as martrix
y = label
from sklearn.naive_bayes import MultinomialNB
clf_NB = MultinomialNB()

clf = MultinomialNB()


In [192]:
clf_NB.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [193]:
# make predictions
expected = label
predicted = clf_NB.predict(X)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          1       0.94      0.90      0.92     13020
          2       0.95      0.99      0.97     12273
          3       0.89      0.89      0.89     11947
          4       0.91      0.92      0.91     12759

avg / total       0.92      0.92      0.92     49999

[[11704   503   545   268]
 [   92 12113    39    29]
 [  366    71 10638   872]
 [  339    59   681 11680]]


In [194]:
clf_NB.fit(X_tfidf, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [195]:
# make predictions
expected = label
predicted = clf_NB.predict(X_tfidf)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          1       0.92      0.88      0.90     13020
          2       0.94      0.98      0.96     12273
          3       0.87      0.86      0.87     11947
          4       0.89      0.89      0.89     12759

avg / total       0.90      0.90      0.90     49999

[[11493   577   618   332]
 [  135 12039    52    47]
 [  458    98 10302  1089]
 [  448   116   814 11381]]


# Classification (svm)

In [196]:
from sklearn import svm
clf_svm = svm.SVC()

In [197]:
clf_svm.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [198]:
# make predictions
expected = label
predicted = clf_svm.predict(X)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          1       0.26      1.00      0.41     13020
          2       0.00      0.00      0.00     12273
          3       1.00      0.00      0.00     11947
          4       0.00      0.00      0.00     12759

avg / total       0.31      0.26      0.11     49999

[[13020     0     0     0]
 [12273     0     0     0]
 [11942     0     5     0]
 [12759     0     0     0]]


In [199]:
clf_svm.fit(X_tfidf, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [200]:
#clf.predict(x)

In [201]:
# make predictions
expected = label
predicted = clf_svm.predict(X_tfidf)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          1       0.26      1.00      0.41     13020
          2       0.00      0.00      0.00     12273
          3       0.00      0.00      0.00     11947
          4       0.00      0.00      0.00     12759

avg / total       0.07      0.26      0.11     49999

[[13020     0     0     0]
 [12273     0     0     0]
 [11947     0     0     0]
 [12759     0     0     0]]


# Testing the model 

# Accuracy on training set

In [205]:
new_text = [" Meanwhile, Idea Cellular on Monday reported a net loss of ₹1236 crore for the second quarter of this fiscal due to free voice calls offered by Reliance Jio, disrupting the Indian mobile telephony market. "]

In [210]:
x = tfvectorizer.transform(new_text).toarray()

In [211]:
find_news_category(x)

@NB : This NEWS is related to : Sci/Tech
@SVM : This NEWS is related to : World


In [212]:
new_text = [" The rise of solar and wind energy has created challenges for GEs power business, which is focused on selling and servicing the turbines and other equipment at the heart of natural gas- and coal-fired plants. GE and Siemens are selling fewer of these systems, raising questions about their ability to capitalize on lucrative service contracts.Our power business is a challenged business right now. Weve got a lot of work to do,said GE CEO John Flannery during a conference call on Monday. "]

In [213]:
x = tfvectorizer.transform(new_text).toarray() 

In [214]:
#print(clf.predict(x))

In [215]:
find_news_category(x)

@NB : This NEWS is related to : Sci/Tech
@SVM : This NEWS is related to : World


In [216]:
new_text = [" If Virat plays in his current form for 5 more years, he will break most of the records made by Sachin Tendulkar. "]

In [217]:
x = tfvectorizer.transform(new_text).toarray() 

In [218]:
find_news_category(x)

@NB : This NEWS is related to : Sports
@SVM : This NEWS is related to : World


# Sentiment Analysis 

In [219]:
from textblob import TextBlob
import sys

In [220]:
def get_tweet_sentiment(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    '''
    # create TextBlob object of passed tweet text
    analysis = TextBlob(tweet)
    # set sentiment
    if analysis.sentiment.polarity >= 0.3:
        return 'positive'
    elif (analysis.sentiment.polarity < 0):
        return 'negative'
    else:
        return 'neutral'

In [221]:
get_tweet_sentiment(new_text[0])

'positive'